In [8]:
import pandas as pd
import nltk
import re
import jieba
import tqdm
from gensim.models import Word2Vec
import pickle

In [9]:
sent = re.sub(r"(\w)'s\b", r"\1 's", "dady's car") 
print(sent)
re.sub(r"(\d+)kgs ", lambda m: m.group() + ' kg ', "4kgs ") 

dady 's car


'4kgs  kg '

In [10]:
### 英文文本正则化预处理
def simple_token(sent):
    # 在正则表达式内部，可以用\n引用括号匹配的内容，n是从1开始的自然数，表示对应顺序的括号
    sent = re.sub(r"(\w)'s\b", r"\1 is", sent)    # make 's a separate word
    sent = re.sub(r"(\w[’'])(\w)", r"\1 \2", sent) # other ' in a word creates 2 words
    sent = re.sub(r"([\"().,;:/_?!-])", r" \1 ", sent).replace('-',' ')    # add spaces around punctuation
    sent = re.sub(r"  *", r" ", sent)               # replace multiple spaces with just one
    return sent.lower()

### 中文文本正则化处理
def zh_token(sent):
    sent = re.sub(r"")

class MySentence():
    def __init__(self, dirname):
        self.dirname = dirname
        
    def __iter__(self):
        for line in tqdm.tqdm(open(self.dirname)):
            yield simple_token(line)
            
class MyZhSentence():
    def __init__(self, dirname):
        self.dirname = dirname
    
    def __iter__(self):
        for line in tqdm.tqdm(open(self.dirname)):
            yield list(jieba.cut(line.strip()))

In [11]:
en_file = "train/train.en"
en_corpus = MySentence(en_file)
for i,sent in enumerate(en_corpus):
    if i > 10:
        break
    print(sent)

0it [00:00, ?it/s]

a pair of red crowned cranes have staked out their nesting territory

a pair of crows had come to nest on our roof as if they had come for lhamo . 

a couple of boys driving around in daddy is car . 

a pair of nines ? you pushed in with a pair of nines ? 

fighting two against one is never ideal , 

it is a neat one two . walker to burton . 

deuces the winner . 

five on one . five on one . yeah , not the greatest odds . 

an incredibly emotional fight between 2 sisters？

one against 500 . 

pair of fives , the winner . 



In [3]:
zh_file = "train/train.zh"
zh_corpus = MyZhSentence(zh_file)
for i,sent in enumerate(zh_corpus):
    if i > 5:
        break
    print(sent)

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.411 seconds.
Prefix dict has been built succesfully.
1it [00:00,  2.42it/s]

['一对', '丹顶鹤', '正', '监视', '着', '它们', '的', '筑巢', '领地']
['一对', '乌鸦', '飞', '到', '我们', '屋顶', '上', '的', '巢里', '，', '它们', '好像', '专门', '为拉木', '而', '来', '的', '。']
['一对', '乖乖仔', '开着', '老爸', '的', '车子', '。']
['一对', '九', '？', '一对', '九', '你', '就', '全', '下注', '了', '？']
['一对二', '总', '不是', '好事', '，']
['一对二', '，', '沃克', '传给', '波顿', '。']


In [4]:
def train_wordvec(lan="en", embedding_dim = 300):
    output = 'vocab/' + lan
    if lan == 'en':
        corpus = MySentence('train/train.en')
    else:
        corpus = MyZhSentence("train/train.zh")
    model = Word2Vec(corpus, size=embedding_dim, min_count=10, sg=1)
    print(lan + "model trained")
    
    # 词表
    vocabulary = model.wv.vocab  # dict
    words = list(vocabulary.keys())
    
    # 开头、结尾、padding、未知词
    function_words = ['<pading>','<start>','<end>','<unk>']
    words = function_words + words    
    word2index = dict(zip(words, range(len(words))))
    
    input_dim = len(words)
    embeddings = []
    for word in vocabulary:
        embeddings.append(model[word])
    embeddings = np.array(embeddings, dtype=np.float32)
    
    weights = np.zeros((input_dim, embedding_dim), np.float32)
    weights[1] = np.ones(embedding_dim, np.float32) * 0.33   # start
    weights[2] = np.ones(embedding_dim, np.float32) * 0.66   # end
    weights[3] = np.average(embeddings, axis = 0)
    weights[4:] = embeddings
    
    # 保存词表
    pickle.dump(word2index, open(output + '_vocab.pkl', 'wb'))
    # 保存词向量
    np.save(output + '_word2vec.npy', embeddings)
    print(lan + '_word vector saved!')

### 训练好的词向量怎么评价质量

In [17]:
import os
os.mknod("en_vocab.pkl")